In [4]:
from ls_helper.models.main_models import ProjectData, get_project

twitter = "twitter"
es = "es"
tw_es = {"platform": twitter, "language": es}
_default = tw_es

po: ProjectData = get_project(**_default)

2025-04-23 16:22:54,886 - ls_helper.models.main_models - INFO - 51/twitter-es-4/twitter/es


In [5]:
ann_res = po.get_raw_annotations_results()

Get recent, gets latest annotation: 0423_1248


In [7]:
from main import agreements
from ls_helper.fresh_agreements import AgreementResult, Agreements
count_vars = ["nature_visual"]

fp, agreements = agreements(**tw_es, variables=count_vars)
agreements: Agreements = agreements
agg: dict[str,AgreementResult] = po.get_agreement_data()

2025-04-23 16:23:59,518 - ls_helper.models.main_models - INFO - 51/twitter-es-4/twitter/es
Get recent, gets latest annotation: 0423_1248
Get recent, gets latest annotation: 0423_1248
2025-04-23 16:24:02,842 - ls_helper.fresh_agreements - INFO - dropped unfinished tasks: 1
2025-04-23 16:24:02,844 - ls_helper.fresh_agreements - DEBUG - dropped tasks: {33891}
nature_visual 8872


KeyError: 'nature_visual'

In [4]:
agg

{'nature_any': AgreementResult(variable='nature_any', single_overall={'gwet': 0.86, 'ratio': 0.9, 'abs': 1003.0, 'total': 1109.0}, options_agreements={'No': {'gwet': 0.87, 'ratio': 0.89, 'abs': 838.0, 'total': 944.0}, 'Yes': {'gwet': 0.43, 'ratio': 0.61, 'abs': 165.0, 'total': 271.0}})}

In [5]:
from ls_helper.my_labelstudio_client.models import AnnotationResult


counts = {}

for task_res in ann_res.task_results:
    found_options = set()
    for ann in task_res.annotations:
        missing_vars = count_vars[:]
        if ann.was_cancelled:
            continue
        for res in ann.result:
            res: AnnotationResult
            variable = res.from_name
            if variable in count_vars:
                options_counts = counts.setdefault(variable, {})
                values = res.value.direct_value
                if not values:
                    found_options.update(["NONE"])
                else:
                    found_options.update(values)
                missing_vars.remove(variable)
        if missing_vars:
            print(f"{task_res.id}/{ann.id}/- {missing_vars}")
    for option in found_options:
        options_counts[option] = options_counts.setdefault(option, 0) + 1




In [6]:
for var,var_opt_counts in counts.items():
    for option,count in var_opt_counts.items():
        if count != (tot_calc := agg[var].options_agreements[option]["total"]):
            print(f"{var}/{option}: calc: {tot_calc}, count: {count}")

In [21]:
# I get: nature_any/Yes: calc: 270.0, count: 271, so lets investigate
nature_any_df = agreements.select_variables(["nature_any"], always_as_dict=False).copy()
nature_any_df.head()
nature_any_df = nature_any_df.value.explode()
nature_any_df_yes = nature_any_df[nature_any_df == "Yes"]
nature_any_df_yes.reset_index()["task_id"].nunique()

271

In [ ]:
# well ok, that also gives us the 271.
# check:
# 1. check unfinished drops tasks? unlikely.. well actually, that was it...!


In [ ]:
occurances = {} #var,option,tuple[task_id,user_id]

for task_res in ann_res.task_results:
    found_options = set()
    for ann in task_res.annotations:
        missing_vars = count_vars[:]
        if ann.was_cancelled:
            continue
        for res in ann.result:
            res: AnnotationResult
            variable = res.from_name
            if variable in count_vars:
                options_counts = counts.setdefault(variable, {})
                values = res.value.direct_value
                if not values:
                    found_options.update(["NONE"])
                else:
                    found_options.update(values)
                missing_vars.remove(variable)
        if missing_vars:
            print(f"{task_res.id}/{ann.id}/- {missing_vars}")
    for option in found_options:
        options_counts[option] = options_counts.setdefault(option, 0) + 1
